# Demonstrating *"chaining"* the binomial option pricing method over an arbitrary timeframe to show how it can approximate Black-Scholes

Consider the following scenario. 

![Example Scenario](img/two_period_example.png)

## Question: What would be the fair price of European style call option that expires 1 year from now?

I'm going to things differently. The answer here is `$ 4.43`. But how did I get to this answer and how do I know that it is correct? Rather than just asking you to blindly trust me, I'll show you my thought process.

## Start with a naive assumption.

Consider answering this question using the single period binomial option pricing method? After all, we know the current stock price, the strike price, the payoffs in the up and down state, and risk free rate. Is it safe to assume that we can focus on just the beginning and the end? Like this? 

![A niave assumption](img/naive_assumption_example.png)

This seems like a promising start. Let's price it anyway using the binomial pricing code I built.

In [1]:
from atop.options.calloption import CallOption

call_calc = CallOption(100, 110, 121, 90.25, .1)
call_calc.print_calc_values()

  INFORMATION FOR A SINGLE PEIORD CALL OPTION USING BINMOMIAL PRICING METHOD

The calculations below are for a single period CALL option where the underlying value is 100, 
with a strike price of 110, an up price of 121, a down price of 90.25, and a risk free rate of 10.0%.
All outputs are rounded to 2 decimal places.  

--------------
Calculations
--------------
Calculated Option Price: 6.42
Calculated Hedge Ratio: 0.36
Calculated present value of bond position: -29.35
Up state payoff is 11 and down state payoff is 0
Calculated risk neutral probability for up state is 0.64 and for down state risk neutral probability is 0.36
_____________________________________________________________________________________________________________



Oh look an answer... it appears the price is ` $6.42`. But wait...

# Looks can be deceiving; *this answer is wrong!*

How do we know that it is wrong? Well, let's think about the assumptions we made here and see if they hold. The implicit assumptions here are first, there are only 2 payoff states and second, the option price doesn't evolve over time. 

## Is it safe to assume that there are only 2 payoff states?

Let's look back to the question. Are there only 2 end payoff states? What does the tree look like again?

![The tree](img/tree_example.png)

If we look all the way to the right we can see that there are actually 3 payoff states, not 2. Since this assumption failed then we know that the answer must be off. We could stop here if we wanted satisfied that `$ 6.42` isn't the correct price. But let's analyze the second assumption as it will become useful in a bit.

## Is it safe to assume that the option value doesn't evolve over time?

That is to say, suppose we paid some price, P for a call option and then the underlying price crashes into the ground such that it is so far low below the strike price that there is zero chance of recovering. Is that option still worth what we paid? NO! I guarentee that it's worthless. In fact if you find someone who is willing to pay anything near what you bought the call option for, please let me know, I've got a bridge I want to sell them. 

Conversely, what if the underlying price skyrockets well above the strike price and we go to sell our option? Is it still worth what we paid for? NO! At the very least, it should be worth the difference in the current underlying price and the strike price. This is especially true for American style call options and European call options that are close to expiration. If you sold it for anything less, then you are essentially giving free money away. 

Rephrased, suppose that we messed up and actually bought a call option at `$ 6.42` on this underlying asset (seriously, I've got a bridge for sale). Does this mean that we can never profit from this transaction? Well, in actuality, our mistake doesn't rule out the possiblity of profit. Which is good news. But it doesn't guarentee that we will make a profit (or fully recover the price we paid.)\*


\* Side question. If possible, at what point will the option price be higher than `$ 6.42`?

# Have we hit a dead end?

Suprisingly, no. We are actually heading in the right direction. There is an unstated assumption we got correct; using the single period binomial option pricing method. Think of the method first and foremost as a tool. Every tool has a time and place. "A screwdriver isn't useful as a hammer." Rather than retool our method, let's change where and when we use it. Before we do that though I want to draw your attention to something and then we will get back to pricing this option.

If you haven't noticed it yet then let me point it out; the tree appears to be several different options *chained* together. Let's seperate out these options. 

### Let's call this the 'up' option.
(Because in the prior period the underlying asset price went up)

![The 'Up' Option](img/up_option_example.png)

### Let's call this the 'down' option.

![The 'Down' Option](img/down_option_example.png)


### Lastly let's call this Today's option.

![The 'Starting' Option](img/start_option_example.png)

When we break out the original tree into distinct single period options, suddenly the tasks looks a lot more solvable. Let's try using the single period binomial option pricing now and see where that gets us.

# Pricing the 'up' option:
I'll save you the trouble of scrolling up and hunting for the values we need for this calculation. Here's what we have:

* Stock Price: `$ 110`
* Strike Price: `$ 110` (that is, currently the underlying price is the same as the strike price).
* up value is `$ 121`
* down value is `$ 104.50`
* and the risk-free rate for the period is `0.05`

In this single period binomial call option the up state is positive, `121 - 110 = 11,` and the down state payoff is negative (`104.50 - 110 =  -5.50`) but this is a call option so we are not obligated to exercise the option, which means the down state payoff is `0`.

Using the `CallOption()` class I wrote, we find the following:

In [2]:
up_opt = CallOption(110, 110, 121, 104.50, 0.05)
up_opt.print_calc_values()

  INFORMATION FOR A SINGLE PEIORD CALL OPTION USING BINMOMIAL PRICING METHOD

The calculations below are for a single period CALL option where the underlying value is 110, 
with a strike price of 110, an up price of 121, a down price of 104.5, and a risk free rate of 5.0%.
All outputs are rounded to 2 decimal places.  

--------------
Calculations
--------------
Calculated Option Price: 6.98
Calculated Hedge Ratio: 0.67
Calculated present value of bond position: -66.35
Up state payoff is 11 and down state payoff is 0
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33
_____________________________________________________________________________________________________________



From the terminal printout we can see that the fair price for the call option at t1 when in the up state is `$ 6.98`. In simple terms, we know that the price of the option will be `$ 6.98` in `6 months` when the underlying price increases to `$ 110`. 

# Pricing the 'down' option:

* Stock Price: `$ 95`
* Strike Price: `$ 110` (that is, currently the underlying price is below the strike price).
* up value is `$ 104.50`
* down value is `$ 90.25`
* and the risk-free rate for the period is `0.05`

Without even running my code I know that the call option's value is worthless as it is already below the strike price and will never rise above the strike price. Don't believe me? Observe:

In [3]:
down_opt = CallOption(95, 110, 104.50, 90.25, 0.05)
down_opt.print_calc_values()

  INFORMATION FOR A SINGLE PEIORD CALL OPTION USING BINMOMIAL PRICING METHOD

The calculations below are for a single period CALL option where the underlying value is 95, 
with a strike price of 110, an up price of 104.5, a down price of 90.25, and a risk free rate of 5.0%.
All outputs are rounded to 2 decimal places.  

--------------
Calculations
--------------
Calculated Option Price: 0.0
Calculated Hedge Ratio: 0.0
Calculated present value of bond position: 0.0
Up state payoff is 0 and down state payoff is 0
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33
_____________________________________________________________________________________________________________



From the terminal printout we can see that the value of the vall option in the down state is... drum roll... `0.00`. 

Great looks like we've got this down. Now we just do the same thing to find today's price, correct?

Nope!

# Pricing the option today: the twist
At this point its common for learners to confuse what to do next. For the past 2 steps we've been calculating these *smaller* single period binomial options. So we do the same for this last one, right?... Well, let's see what we get out of it when we calculate the option following the same approach from before... ...if you haven't seen the issue yet, let me paint it to you using the following picture. 

![Big Issue](img/big_issue_example.png)

The value of this simple call option will be zero. The underlying price will never rise to above the strike price of `$ 110`. No one would exercise this call option. Therefor, under this analysis, the value of the option is worthless. 

But at the start of this document I stated that the answer is `$ 4.43`. How do we reconcile this discrepancy. The approach we've been using generated a price of zero, whereas I've been saying the actual price for this example is `4.43`. Again we are presented with the question, "how do we know when the answer is wrong?" When we criticaly think through the most recent step we will see that we made a number of logical errors.

### What we did wrong:

We assumed that the tool used before could also work here. We saw that the past 2 steps each followed the exact same approach pattern. It's not a stretch to assume this pattern continues forward. However, consider that there is no logical reason for the persistence of this pattern. It might very well be that by chance the first 2 steps followed the same approach. (This will give rise to an important corollary later). In short, we've fallen victim to a "monkey see, monkey do" mindset. We are better than that. 

Remember that we found the fair price for this call option 6 months from today when the underlying asset's price rises to `$ 110` and similarly we found the fair price when the underlying asset's price falls to `$ 95`. The call option's prices are `$ 6.98` and `$ 0.00` respectively. Are we to just ignore these values? No! But our approach took one look at those calculations and went "oh... *guess we don't need these!*" and threw them out. But these are the payoffs of the call option if we were to sell it in 6 months! The work from earlier demonstrated that the option has some intrinsic value when the underlying price rises to `$ 110`. This value in 6 months must carry in part back to today. Else, we could pay nothing for the option today, and pocket $6.98 in 6 months! What a great deal! We would make money from nothing! This illogical situation suggests that 1). the call option today must be worth some positive value, and 2) the call option's value is somewhere between `$ 0.00` and `$ 6.98`. By this logic we must reject that the call option has zero value today!

Let's put some numbers to it for example. Suppose we mispriced our calculation and bought the option today at exactly `$ 6.00`. There is still a chance that we can profit from this mistake if the underlying price rises to `$ 110` in 6 months which would push the fair market price of the option to `$ 6.98`. We could sell our option at that `6.98` and walk away with `$ 0.98` in profit. (which is the answer to that side question from earlier). Conversely if the underlying price instead falls to `$ 95` we are under no obligation to exercise our option, in which case our profit is zero. It's these payoff states we care about. Recall that these are a vital component in the calculating the fair price of an option.

In the single period binomial option module the payoffs are normally calculated internally but we can override those payoff calculations by passing additional arguments. (Dev note: I'm contemplating moving this feature to its own module in the future, I've opted to bundle this functionality together for now.) The following demonstrates this and produces the answer we've been looking for. 

In [7]:
todays_option = CallOption(100, 110, 110, 95, 0.05, 6.98, 0)
todays_option.print_calc_values()

  INFORMATION FOR A SINGLE PEIORD CALL OPTION USING BINMOMIAL PRICING METHOD

The calculations below are for a single period CALL option where the underlying value is 100, 
with a strike price of 110, an up price of 110, a down price of 95, and a risk free rate of 5.0%.
All outputs are rounded to 2 decimal places.  
USER HAS OPTED TO OVERRIDE PAYOFF VALUES! BE AWARE THIS WILL AFFECT CALCULATIONS!

--------------
Calculations
--------------
Calculated Option Price: 4.43
Calculated Hedge Ratio: 0.47
Calculated present value of bond position: -42.1
Up state payoff is 6.98 and down state payoff is 0
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33
_____________________________________________________________________________________________________________



Finally, there is the answer.

---


# Chaining it all together:
The code below is everything above condensed to a few lines.

In [2]:
up_opt = CallOption(110, 110, 121, 104.50, 0.05)
down_opt = CallOption(95, 110, 104.50, 90.25, 0.05)
start_opt = CallOption(100, 110, 110, 95, 0.05, up_opt.option_price, down_opt.option_price)
start_opt.print_calc_values()

  INFORMATION FOR A SINGLE PEIORD CALL OPTION USING BINMOMIAL PRICING METHOD

The calculations below are for a single period CALL option where the underlying value is 100, 
with a strike price of 110, an up price of 110, a down price of 95, and a risk free rate of 5.0%.
All outputs are rounded to 2 decimal places.  
USER HAS OPTED TO OVERRIDE PAYOFF VALUES! BE AWARE THIS WILL AFFECT CALCULATIONS!

--------------
Calculations
--------------
Calculated Option Price: 4.43
Calculated Hedge Ratio: 0.47
Calculated present value of bond position: -42.13
Up state payoff is 6.984126984126988 and down state payoff is 0
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33
_____________________________________________________________________________________________________________



---
# Generalizing the approach

There is no limit to the number of periods we can have to describe the evolution of the underlying price and thus how accurate we can be with our calculation. But we must keep in mind a few things:
* Today's price is a reflection of the next time period's payoffs, and those payoffs are a reflection of the next time period's payoff, etc. 
* How the problem describes volatility. Is the volatility described in annual terms or in terms of periods.
* How the problem describes the risk-free rate. 

Consider the following general example with 4 distinct time periods:

![general binomial tree](img/four_period_general.png)

Between each time period, the underlying price can move up or down by some factor, u or d, respectively. For instance, suppose in each time period the volatitily of the underlying can be described as either increasing by 5% or decreasing by 5%. In the calculation, the up factor would be `u = (1 + 0.05) = 1.05` and the down factor is `d = (1 + -0.05) = 0.95`. In the case of only increases for t4, the stock price has increase by 4 factors of u. Stated differently, the highest possible underlying price is u\*u\*u\*u\*S = (u^4) \* S.  In the case of 2 ups and 2 downs is u\*u\*d\*d\*S = (u^2) \* (d^2) \* S. 

### You can imagine any possible binomial tree with n-periods. 
